# Enhanced Farmer Simulation with Graph-Based Social Networks

This notebook extends the original simulation by integrating NetworkX graph structures to model:
- Social networks between farmers
- Knowledge diffusion through networks
- Network-influenced decision making
- Different network topologies (small-world, scale-free, etc.)

In [7]:
import json
import random
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.datacollection import DataCollector

try:
    import seaborn as sns
    sns.set(style="whitegrid")
except ImportError:
    sns = None

# Load data files
with open("./data/zones.json") as f:
    zones_data = json.load(f)

with open("./data/crops.json") as f:
    crops_data = json.load(f)

with open("./data/climate_profiles.json") as f:
    climate_data = json.load(f)

with open("./data/soil_moisture_profiles.json") as f:
    soil_moisture = json.load(f)

In [8]:
class Zone:
    def __init__(self, zone_id, data, climate_profile, soil_moisture):
        self.id = zone_id
        self.name = data["name"]
        self.soil_types = data["soil_types"]
        self.dominant_soil = data["dominant_soil"]
        self.viable_crops = data["viable_crops"]
        self.agriculture_type = data["agriculture_type"]
        self.climate_profile = climate_profile
        
        # Soil moisture
        sm_data = soil_moisture["zones"][zone_id]
        self.mean_sm = sm_data["mean"]
        self.std_sm = sm_data["std"]
        
        # Shared knowledge at zone level
        self.shared_crop_yield = {}
        self.shared_crop_profit = {}

In [9]:
class FarmerAgent(Agent):
    """
    Enhanced Farmer Agent with graph-based social network capabilities.
    """
    
    def __init__(self, model, unique_id, zone, land_size, strategy_type, 
                 innovativeness=0.5, risk_tolerance=0.5):
        super().__init__(model)
        self.unique_id = unique_id
        self.zone = zone
        self.land_size = land_size
        self.strategy_type = strategy_type  # "INDIVIDUAL", "SHARED", or "NETWORK"
        
        # Personal characteristics
        self.innovativeness = innovativeness  # 0-1: willingness to try new crops
        self.risk_tolerance = risk_tolerance  # 0-1: tolerance for risky decisions
        
        # Memory and performance tracking
        self.memory = []  # Historical performance
        self.chosen_crop = None
        self.yield_t_ha = 0
        self.profit = 0
        
        # Network-based attributes
        self.neighbors_influence = {}  # Track influence from each neighbor
        self.crop_observations = {}  # What crops neighbors are growing
        self.network_trust = {}  # Trust level for each neighbor (0-1)
        
    def step(self):
        """Execute one time step for the farmer."""
        # First, observe what neighbors are doing
        if self.strategy_type == "NETWORK":
            self.observe_neighbors()
        
        # Make crop decision
        self.decide_crop()
        
        # Compute yield and profit
        self.compute_yield()
        
        # Update network trust based on performance
        if self.strategy_type == "NETWORK":
            self.update_network_trust()
    
    def observe_neighbors(self):
        """Observe what crops neighbors are growing and their performance."""
        if not hasattr(self.model, 'social_network'):
            return
        
        self.crop_observations = {}
        neighbors = list(self.model.social_network.neighbors(self.unique_id))
        
        for neighbor_id in neighbors:
            neighbor = self.model.get_agent(neighbor_id)
            if neighbor and neighbor.chosen_crop:
                if neighbor.chosen_crop not in self.crop_observations:
                    self.crop_observations[neighbor.chosen_crop] = []
                
                # Record neighbor's crop and performance
                self.crop_observations[neighbor.chosen_crop].append({
                    'neighbor_id': neighbor_id,
                    'yield': neighbor.yield_t_ha,
                    'profit': neighbor.profit,
                    'trust': self.network_trust.get(neighbor_id, 0.5)
                })
    
    def update_network_trust(self):
        """Update trust in neighbors based on their performance vs. yours."""
        if not hasattr(self.model, 'social_network'):
            return
        
        neighbors = list(self.model.social_network.neighbors(self.unique_id))
        
        for neighbor_id in neighbors:
            neighbor = self.model.get_agent(neighbor_id)
            if not neighbor:
                continue
            
            # Initialize trust if not present
            if neighbor_id not in self.network_trust:
                self.network_trust[neighbor_id] = 0.5
            
            # Update trust based on relative performance
            if self.profit > 0 and neighbor.profit > 0:
                performance_ratio = neighbor.profit / self.profit
                # Gradually adjust trust towards performance ratio
                learning_rate = 0.1
                target_trust = min(1.0, performance_ratio)
                self.network_trust[neighbor_id] += learning_rate * (target_trust - self.network_trust[neighbor_id])
                self.network_trust[neighbor_id] = np.clip(self.network_trust[neighbor_id], 0.1, 1.0)
    
    def decide_crop(self):
        """Decide which crop to plant based on strategy type."""
        zone = self.zone
        suitability_scores = {}
        
        # Base suitability calculation for all crops
        for crop in zone.viable_crops:
            suitability_scores[crop] = self.compute_suitability(crop)
        
        # Apply strategy-specific modifications
        if self.strategy_type == "INDIVIDUAL":
            # Pure individual decision based on own experience
            suitability_scores = self._apply_individual_learning(suitability_scores)
            
        elif self.strategy_type == "SHARED":
            # Use zone-level shared knowledge
            suitability_scores = self._apply_shared_knowledge(suitability_scores)
            
        elif self.strategy_type == "NETWORK":
            # Use network-based social learning
            suitability_scores = self._apply_network_influence(suitability_scores)
        
        # Choose crop (with some randomness based on innovativeness)
        if random.random() < self.innovativeness * 0.2:  # 20% chance to explore at max innovativeness
            # Explore: choose randomly from viable crops
            self.chosen_crop = random.choice(zone.viable_crops)
        else:
            # Exploit: choose best crop based on scores
            self.chosen_crop = max(suitability_scores, key=suitability_scores.get)
    
    def _apply_individual_learning(self, scores):
        """Modify scores based on individual past experience."""
        if len(self.memory) > 0:
            # Build experience record
            crop_performance = {}
            for mem in self.memory[-5:]:  # Last 5 seasons
                crop = mem.get('crop')
                if crop:
                    if crop not in crop_performance:
                        crop_performance[crop] = []
                    crop_performance[crop].append(mem.get('profit', 0))
            
            # Adjust scores based on experience
            for crop in scores:
                if crop in crop_performance:
                    avg_profit = np.mean(crop_performance[crop])
                    max_profit = max([np.mean(v) for v in crop_performance.values()] + [1])
                    experience_boost = (avg_profit / max_profit) if max_profit > 0 else 0
                    scores[crop] = 0.6 * scores[crop] + 0.4 * experience_boost
        
        return scores
    
    def _apply_shared_knowledge(self, scores):
        """Modify scores based on zone-level shared knowledge."""
        if hasattr(self.zone, "shared_crop_profit") and self.zone.shared_crop_profit:
            max_shared = max(self.zone.shared_crop_profit.values())
            for crop in scores:
                if crop in self.zone.shared_crop_profit and max_shared > 0:
                    shared_signal = self.zone.shared_crop_profit[crop] / max_shared
                    scores[crop] = 0.5 * scores[crop] + 0.5 * shared_signal
        
        return scores
    
    def _apply_network_influence(self, scores):
        """Modify scores based on network neighbors' experiences."""
        if not self.crop_observations:
            # Fallback to shared knowledge if no observations yet
            return self._apply_shared_knowledge(scores)
        
        # Calculate network-weighted performance for each crop
        network_scores = {}
        for crop in scores:
            if crop in self.crop_observations:
                observations = self.crop_observations[crop]
                # Weighted average based on trust
                weighted_profit = sum(obs['profit'] * obs['trust'] for obs in observations)
                total_trust = sum(obs['trust'] for obs in observations)
                
                if total_trust > 0:
                    network_scores[crop] = weighted_profit / total_trust
        
        # Normalize and combine with base suitability
        if network_scores:
            max_network = max(network_scores.values())
            if max_network > 0:
                for crop in scores:
                    network_signal = network_scores.get(crop, 0) / max_network
                    # Weight combination based on risk tolerance
                    # High risk tolerance = more weight on base suitability (explore)
                    # Low risk tolerance = more weight on network (follow others)
                    base_weight = 0.3 + 0.4 * self.risk_tolerance
                    network_weight = 1.0 - base_weight
                    scores[crop] = base_weight * scores[crop] + network_weight * network_signal
        
        return scores
    
    def compute_suitability(self, crop_name):
        """Compute base suitability of a crop for the farmer's land."""
        crop = crops_data["crops"][crop_name]
        zone = self.zone
        score = 0.0
        weight_total = 0.0
        
        # Zone compatibility
        zone_weight = 0.3
        if zone.id in crop["ideal_zones"]:
            score += 1.0 * zone_weight
        elif zone.id in crop.get("suitable_zones", []):
            score += 0.6 * zone_weight
        weight_total += zone_weight
        
        # Soil compatibility
        soil_weight = 0.25
        ideal_soils = crop.get("ideal_soil_types", [])
        if zone.dominant_soil in ideal_soils:
            score += 1.0 * soil_weight
        elif any(s in ideal_soils for s in zone.soil_types):
            score += 0.7 * soil_weight
        weight_total += soil_weight
        
        # Soil moisture
        sm_weight = 0.2
        sm_range = crop.get("soil_moisture_range", [0, 100])
        if sm_range[0] <= zone.mean_sm <= sm_range[1]:
            # Perfect match
            score += 1.0 * sm_weight
        else:
            # Partial penalty based on distance from range
            if zone.mean_sm < sm_range[0]:
                gap = sm_range[0] - zone.mean_sm
            else:
                gap = zone.mean_sm - sm_range[1]
            penalty = max(0, 1.0 - gap / 20.0)
            score += penalty * sm_weight
        weight_total += sm_weight
        
        # Climate (temperature and precipitation)
        climate_weight = 0.25
        temp_range = crop.get("temperature_range", [0, 50])
        precip_range = crop.get("precipitation_range", [0, 3000])
        
        # Get current climate from zone
        climate = zone.climate_profile
        current_temp = climate.get("mean_temp", 20)
        current_precip = climate.get("annual_precip", 500)
        
        temp_score = 1.0 if temp_range[0] <= current_temp <= temp_range[1] else 0.5
        precip_score = 1.0 if precip_range[0] <= current_precip <= precip_range[1] else 0.5
        
        score += (temp_score + precip_score) / 2.0 * climate_weight
        weight_total += climate_weight
        
        # Normalize
        return score / weight_total if weight_total > 0 else 0.5
    
    def compute_yield(self):
        """Compute crop yield and profit for this season."""
        if not self.chosen_crop:
            self.yield_t_ha = 0
            self.profit = 0
            return
        
        crop = crops_data["crops"][self.chosen_crop]
        base_yield = crop.get("avg_yield_t_ha", 2.0)
        
        # Compute yield multiplier based on suitability
        suitability = self.compute_suitability(self.chosen_crop)
        
        # Add some randomness (weather, pests, etc.)
        noise = random.gauss(1.0, 0.15)
        noise = max(0.5, min(1.5, noise))  # Clamp between 0.5 and 1.5
        
        # Final yield
        self.yield_t_ha = base_yield * suitability * noise
        
        # Compute profit
        price_per_ton = crop.get("price_per_ton", 200)
        cost_per_ha = crop.get("cost_per_ha", 100)
        
        revenue = self.yield_t_ha * price_per_ton * self.land_size
        cost = cost_per_ha * self.land_size
        self.profit = revenue - cost
        
        # Record in memory
        self.memory.append({
            'step': self.model.schedule.steps,
            'crop': self.chosen_crop,
            'yield': self.yield_t_ha,
            'profit': self.profit,
            'suitability': suitability
        })
        
        # Update zone-level shared knowledge
        if self.chosen_crop not in self.zone.shared_crop_yield:
            self.zone.shared_crop_yield[self.chosen_crop] = []
            self.zone.shared_crop_profit[self.chosen_crop] = []
        
        self.zone.shared_crop_yield[self.chosen_crop].append(self.yield_t_ha)
        self.zone.shared_crop_profit[self.chosen_crop].append(self.profit)
        
        # Keep only recent history
        if len(self.zone.shared_crop_yield[self.chosen_crop]) > 20:
            self.zone.shared_crop_yield[self.chosen_crop] = \
                self.zone.shared_crop_yield[self.chosen_crop][-20:]
            self.zone.shared_crop_profit[self.chosen_crop] = \
                self.zone.shared_crop_profit[self.chosen_crop][-20:]
        
        # Update zone averages
        for crop in self.zone.shared_crop_yield:
            self.zone.shared_crop_yield[crop] = np.mean(self.zone.shared_crop_yield[crop])
            self.zone.shared_crop_profit[crop] = np.mean(self.zone.shared_crop_profit[crop])

In [10]:
class FarmingModel(Model):
    """
    Enhanced Farming Model with integrated social network.
    """
    
    def __init__(self, n_farmers=100, network_type='small_world', 
                 network_params=None, strategy_distribution=None):
        super().__init__()
        self.n_farmers = n_farmers
        self.schedule = mesa.activation.RandomActivation(self)
        self.network_type = network_type
        
        # Default strategy distribution: 40% NETWORK, 30% SHARED, 30% INDIVIDUAL
        if strategy_distribution is None:
            strategy_distribution = {
                'NETWORK': 0.4,
                'SHARED': 0.3,
                'INDIVIDUAL': 0.3
            }
        
        # Initialize zones
        self.zones = {}
        for zone_id, zone_data in zones_data["zones"].items():
            climate_profile = climate_data["profiles"].get(zone_id, {})
            self.zones[zone_id] = Zone(zone_id, zone_data, climate_profile, soil_moisture)
        
        # Create farmers
        self.farmers = []
        for i in range(n_farmers):
            # Randomly assign zone
            zone_id = random.choice(list(self.zones.keys()))
            zone = self.zones[zone_id]
            
            # Random land size (1-20 hectares)
            land_size = random.uniform(1, 20)
            
            # Assign strategy based on distribution
            rand = random.random()
            cumulative = 0
            strategy = 'NETWORK'
            for strat, prob in strategy_distribution.items():
                cumulative += prob
                if rand < cumulative:
                    strategy = strat
                    break
            
            # Random personal characteristics
            innovativeness = random.uniform(0.2, 0.8)
            risk_tolerance = random.uniform(0.2, 0.8)
            
            farmer = FarmerAgent(
                model=self,
                unique_id=i,
                zone=zone,
                land_size=land_size,
                strategy_type=strategy,
                innovativeness=innovativeness,
                risk_tolerance=risk_tolerance
            )
            
            self.farmers.append(farmer)
            self.schedule.add(farmer)
        
        # Create social network
        self.social_network = self._create_network(network_type, network_params)
        
        # Data collection
        self.datacollector = DataCollector(
            model_reporters={
                "Mean Profit": lambda m: np.mean([a.profit for a in m.farmers]),
                "Mean Yield": lambda m: np.mean([a.yield_t_ha for a in m.farmers]),
                "Network Profit": lambda m: np.mean([a.profit for a in m.farmers if a.strategy_type == 'NETWORK']),
                "Shared Profit": lambda m: np.mean([a.profit for a in m.farmers if a.strategy_type == 'SHARED']),
                "Individual Profit": lambda m: np.mean([a.profit for a in m.farmers if a.strategy_type == 'INDIVIDUAL']),
            },
            agent_reporters={
                "Profit": "profit",
                "Yield": "yield_t_ha",
                "Strategy": "strategy_type",
                "Crop": "chosen_crop"
            }
        )
    
    def _create_network(self, network_type, params):
        """Create social network based on specified type."""
        n = self.n_farmers
        
        if network_type == 'small_world':
            # Watts-Strogatz small-world network
            k = params.get('k', 4) if params else 4  # Each node connected to k nearest neighbors
            p = params.get('p', 0.1) if params else 0.1  # Rewiring probability
            G = nx.watts_strogatz_graph(n, k, p, seed=42)
            
        elif network_type == 'scale_free':
            # Barabási-Albert scale-free network
            m = params.get('m', 3) if params else 3  # Number of edges to attach from new node
            G = nx.barabasi_albert_graph(n, m, seed=42)
            
        elif network_type == 'random':
            # Erdős-Rényi random graph
            p = params.get('p', 0.05) if params else 0.05  # Probability of edge creation
            G = nx.erdos_renyi_graph(n, p, seed=42)
            
        elif network_type == 'geographic':
            # Geographic network: farmers in same zone are more likely to connect
            G = nx.Graph()
            G.add_nodes_from(range(n))
            
            for i in range(n):
                for j in range(i+1, n):
                    # Higher connection probability for same zone
                    if self.farmers[i].zone.id == self.farmers[j].zone.id:
                        if random.random() < 0.15:  # 15% chance within zone
                            G.add_edge(i, j)
                    else:
                        if random.random() < 0.02:  # 2% chance across zones
                            G.add_edge(i, j)
        
        elif network_type == 'complete':
            # Complete graph (everyone connected)
            G = nx.complete_graph(n)
            
        else:
            # Default: small-world
            G = nx.watts_strogatz_graph(n, 4, 0.1, seed=42)
        
        return G
    
    def get_agent(self, unique_id):
        """Get agent by unique_id."""
        for agent in self.farmers:
            if agent.unique_id == unique_id:
                return agent
        return None
    
    def step(self):
        """Advance the model by one step."""
        self.datacollector.collect(self)
        self.schedule.step()

## Network Visualization and Analysis

In [11]:
def visualize_network(model, show_performance=True):
    """
    Visualize the social network with node colors representing strategies or performance.
    """
    G = model.social_network
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # Layout
    pos = nx.spring_layout(G, seed=42, k=0.5, iterations=50)
    
    # Plot 1: Color by strategy
    ax = axes[0]
    strategy_colors = {'NETWORK': 'blue', 'SHARED': 'green', 'INDIVIDUAL': 'red'}
    node_colors = [strategy_colors[model.farmers[i].strategy_type] for i in G.nodes()]
    
    nx.draw(G, pos, node_color=node_colors, node_size=100, 
            edge_color='gray', alpha=0.6, width=0.5, ax=ax)
    ax.set_title('Social Network - Colored by Strategy', fontsize=14, fontweight='bold')
    
    # Legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='blue', label='NETWORK'),
                      Patch(facecolor='green', label='SHARED'),
                      Patch(facecolor='red', label='INDIVIDUAL')]
    ax.legend(handles=legend_elements, loc='upper right')
    
    # Plot 2: Color by performance (profit)
    if show_performance:
        ax = axes[1]
        profits = [model.farmers[i].profit for i in G.nodes()]
        
        # Normalize for colormap
        from matplotlib import cm
        from matplotlib.colors import Normalize
        
        if max(profits) > min(profits):
            norm = Normalize(vmin=min(profits), vmax=max(profits))
            cmap = cm.RdYlGn  # Red (low) to Green (high)
            node_colors_perf = [cmap(norm(p)) for p in profits]
        else:
            node_colors_perf = 'gray'
        
        nx.draw(G, pos, node_color=node_colors_perf, node_size=100,
                edge_color='gray', alpha=0.6, width=0.5, ax=ax)
        ax.set_title('Social Network - Colored by Profit', fontsize=14, fontweight='bold')
        
        # Colorbar
        sm = cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label('Profit', rotation=270, labelpad=20)
    
    plt.tight_layout()
    plt.show()

def analyze_network_properties(model):
    """
    Analyze and print key network properties.
    """
    G = model.social_network
    
    print("\n=== Network Properties ===")
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print(f"Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
    print(f"Network density: {nx.density(G):.4f}")
    
    if nx.is_connected(G):
        print(f"Average shortest path length: {nx.average_shortest_path_length(G):.2f}")
        print(f"Network diameter: {nx.diameter(G)}")
    else:
        print("Network is not connected")
        components = list(nx.connected_components(G))
        print(f"Number of connected components: {len(components)}")
        print(f"Largest component size: {len(max(components, key=len))}")
    
    print(f"Clustering coefficient: {nx.average_clustering(G):.4f}")
    
    # Degree centrality
    degree_centrality = nx.degree_centrality(G)
    print(f"Average degree centrality: {np.mean(list(degree_centrality.values())):.4f}")
    
    # Find most connected farmers
    top_connected = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
    print("\nTop 5 most connected farmers:")
    for farmer_id, centrality in top_connected:
        farmer = model.farmers[farmer_id]
        print(f"  Farmer {farmer_id}: {G.degree(farmer_id)} connections, "
              f"strategy={farmer.strategy_type}, profit={farmer.profit:.2f}")

## Run Simulation and Compare Network Types

In [12]:
# Run simulation with small-world network
print("Creating model with small-world network...")
model = FarmingModel(
    n_farmers=100,
    network_type='small_world',
    network_params={'k': 6, 'p': 0.1},
    strategy_distribution={'NETWORK': 0.5, 'SHARED': 0.3, 'INDIVIDUAL': 0.2}
)

print("Running simulation for 20 steps...")
for i in range(20):
    model.step()
    if (i + 1) % 5 == 0:
        print(f"Step {i+1}/20 completed")

print("\nSimulation complete!")

# Analyze network
analyze_network_properties(model)

# Visualize network
visualize_network(model, show_performance=True)

Creating model with small-world network...


AttributeError: module 'mesa' has no attribute 'activation'

## Performance Analysis

In [ ]:
# Extract data
model_data = model.datacollector.get_model_vars_dataframe()
agent_data = model.datacollector.get_agent_vars_dataframe()

# Plot model-level metrics over time
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Overall mean profit and yield
ax = axes[0, 0]
model_data['Mean Profit'].plot(ax=ax, color='green', linewidth=2)
ax.set_title('Mean Profit Over Time', fontweight='bold')
ax.set_xlabel('Step')
ax.set_ylabel('Profit')
ax.grid(True, alpha=0.3)

ax = axes[0, 1]
model_data['Mean Yield'].plot(ax=ax, color='blue', linewidth=2)
ax.set_title('Mean Yield Over Time', fontweight='bold')
ax.set_xlabel('Step')
ax.set_ylabel('Yield (t/ha)')
ax.grid(True, alpha=0.3)

# Strategy comparison
ax = axes[1, 0]
model_data[['Network Profit', 'Shared Profit', 'Individual Profit']].plot(ax=ax, linewidth=2)
ax.set_title('Profit by Strategy Over Time', fontweight='bold')
ax.set_xlabel('Step')
ax.set_ylabel('Profit')
ax.legend(['Network', 'Shared', 'Individual'])
ax.grid(True, alpha=0.3)

# Final distribution
ax = axes[1, 1]
final_step = agent_data.index.get_level_values('Step').max()
final_data = agent_data.xs(final_step, level='Step')

if sns:
    sns.boxplot(data=final_data, x='Strategy', y='Profit', ax=ax)
else:
    final_data.boxplot(column='Profit', by='Strategy', ax=ax)
ax.set_title('Final Profit Distribution by Strategy', fontweight='bold')
ax.set_xlabel('Strategy')
ax.set_ylabel('Profit')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n=== Final Performance Summary ===")
summary = final_data.groupby('Strategy')['Profit'].agg(['mean', 'std', 'min', 'max'])
print(summary)

## Compare Different Network Topologies

In [ ]:
def run_network_comparison(steps=20, n_farmers=100):
    """
    Compare performance across different network topologies.
    """
    network_configs = {
        'Small World': ('small_world', {'k': 6, 'p': 0.1}),
        'Scale Free': ('scale_free', {'m': 3}),
        'Random': ('random', {'p': 0.05}),
        'Geographic': ('geographic', None),
    }
    
    results = {}
    
    for name, (net_type, params) in network_configs.items():
        print(f"\nRunning {name} network...")
        model = FarmingModel(
            n_farmers=n_farmers,
            network_type=net_type,
            network_params=params,
            strategy_distribution={'NETWORK': 0.5, 'SHARED': 0.3, 'INDIVIDUAL': 0.2}
        )
        
        for i in range(steps):
            model.step()
        
        # Collect results
        model_data = model.datacollector.get_model_vars_dataframe()
        agent_data = model.datacollector.get_agent_vars_dataframe()
        final_step = agent_data.index.get_level_values('Step').max()
        final_data = agent_data.xs(final_step, level='Step')
        
        results[name] = {
            'model_data': model_data,
            'final_profits': final_data.groupby('Strategy')['Profit'].mean().to_dict(),
            'network': model.social_network
        }
    
    return results

# Run comparison
print("Comparing different network topologies...")
comparison_results = run_network_comparison(steps=20, n_farmers=100)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

network_names = list(comparison_results.keys())

# Plot mean profit over time for each network
ax = axes[0, 0]
for name in network_names:
    data = comparison_results[name]['model_data']
    data['Mean Profit'].plot(ax=ax, label=name, linewidth=2)
ax.set_title('Mean Profit by Network Type', fontweight='bold')
ax.set_xlabel('Step')
ax.set_ylabel('Profit')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot NETWORK strategy performance by network type
ax = axes[0, 1]
for name in network_names:
    data = comparison_results[name]['model_data']
    data['Network Profit'].plot(ax=ax, label=name, linewidth=2)
ax.set_title('NETWORK Strategy Profit by Network Type', fontweight='bold')
ax.set_xlabel('Step')
ax.set_ylabel('Profit')
ax.legend()
ax.grid(True, alpha=0.3)

# Bar chart of final profits
ax = axes[1, 0]
strategies = ['NETWORK', 'SHARED', 'INDIVIDUAL']
x = np.arange(len(strategies))
width = 0.2

for i, name in enumerate(network_names):
    profits = [comparison_results[name]['final_profits'].get(s, 0) for s in strategies]
    ax.bar(x + i*width, profits, width, label=name)

ax.set_xlabel('Strategy')
ax.set_ylabel('Final Mean Profit')
ax.set_title('Final Profit by Strategy and Network Type', fontweight='bold')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(strategies)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Network properties comparison
ax = axes[1, 1]
properties = []
for name in network_names:
    G = comparison_results[name]['network']
    avg_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()
    clustering = nx.average_clustering(G)
    properties.append([avg_degree, clustering])

properties = np.array(properties).T
x = np.arange(len(network_names))
width = 0.35

ax.bar(x - width/2, properties[0], width, label='Avg Degree')
ax.bar(x + width/2, properties[1] * 10, width, label='Clustering × 10')
ax.set_ylabel('Value')
ax.set_title('Network Properties Comparison', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(network_names, rotation=45)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print summary
print("\n=== Network Topology Comparison Summary ===")
for name in network_names:
    print(f"\n{name}:")
    for strategy, profit in comparison_results[name]['final_profits'].items():
        print(f"  {strategy}: {profit:.2f}")

## Analyze Network Influence Dynamics

In [ ]:
def analyze_trust_dynamics(model):
    """
    Analyze how trust evolves in the network.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Collect trust data
    trust_values = []
    for farmer in model.farmers:
        if farmer.strategy_type == 'NETWORK' and farmer.network_trust:
            trust_values.extend(farmer.network_trust.values())
    
    if trust_values:
        # Distribution of trust values
        ax = axes[0]
        ax.hist(trust_values, bins=20, edgecolor='black', alpha=0.7)
        ax.set_xlabel('Trust Level')
        ax.set_ylabel('Frequency')
        ax.set_title('Distribution of Trust Values in Network', fontweight='bold')
        ax.axvline(np.mean(trust_values), color='red', linestyle='--', 
                   linewidth=2, label=f'Mean: {np.mean(trust_values):.2f}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Trust vs Performance correlation
        ax = axes[1]
        farmer_avg_trust = []
        farmer_profits = []
        
        for farmer in model.farmers:
            if farmer.strategy_type == 'NETWORK' and farmer.network_trust:
                avg_trust = np.mean(list(farmer.network_trust.values()))
                farmer_avg_trust.append(avg_trust)
                farmer_profits.append(farmer.profit)
        
        ax.scatter(farmer_avg_trust, farmer_profits, alpha=0.6)
        ax.set_xlabel('Average Trust in Neighbors')
        ax.set_ylabel('Profit')
        ax.set_title('Trust vs Performance (NETWORK farmers)', fontweight='bold')
        
        # Add trend line
        if len(farmer_avg_trust) > 1:
            z = np.polyfit(farmer_avg_trust, farmer_profits, 1)
            p = np.poly1d(z)
            ax.plot(sorted(farmer_avg_trust), p(sorted(farmer_avg_trust)), 
                   "r--", alpha=0.8, linewidth=2, label='Trend')
            ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Run analysis on the model
analyze_trust_dynamics(model)

## Crop Diversity Analysis

In [ ]:
def analyze_crop_diversity(model):
    """
    Analyze crop diversity across strategies.
    """
    from collections import Counter
    
    # Count crops by strategy
    crop_counts = {'NETWORK': Counter(), 'SHARED': Counter(), 'INDIVIDUAL': Counter()}
    
    for farmer in model.farmers:
        if farmer.chosen_crop:
            crop_counts[farmer.strategy_type][farmer.chosen_crop] += 1
    
    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for idx, strategy in enumerate(['NETWORK', 'SHARED', 'INDIVIDUAL']):
        ax = axes[idx]
        counts = crop_counts[strategy]
        
        if counts:
            crops = list(counts.keys())
            values = list(counts.values())
            
            ax.bar(range(len(crops)), values, color=
                  {'NETWORK': 'blue', 'SHARED': 'green', 'INDIVIDUAL': 'red'}[strategy],
                  alpha=0.7)
            ax.set_xticks(range(len(crops)))
            ax.set_xticklabels(crops, rotation=45, ha='right')
            ax.set_ylabel('Number of Farmers')
            ax.set_title(f'{strategy} Strategy\n({len(crops)} different crops)', 
                        fontweight='bold')
            ax.grid(True, alpha=0.3, axis='y')
            
            # Add diversity metric
            total = sum(values)
            diversity = -sum((v/total) * np.log(v/total) for v in values if v > 0)
            ax.text(0.95, 0.95, f'Shannon Index: {diversity:.2f}',
                   transform=ax.transAxes, ha='right', va='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\n=== Crop Diversity Summary ===")
    for strategy in ['NETWORK', 'SHARED', 'INDIVIDUAL']:
        counts = crop_counts[strategy]
        print(f"\n{strategy}:")
        print(f"  Number of different crops: {len(counts)}")
        if counts:
            most_common = counts.most_common(3)
            print("  Top 3 crops:")
            for crop, count in most_common:
                print(f"    {crop}: {count} farmers")

analyze_crop_diversity(model)

## Export Results

In [ ]:
# Export model data
model_data = model.datacollector.get_model_vars_dataframe()
model_data.to_csv('model_results.csv')

# Export agent data
agent_data = model.datacollector.get_agent_vars_dataframe()
agent_data.to_csv('agent_results.csv')

# Export network structure
import json
network_data = nx.node_link_data(model.social_network)
with open('network_structure.json', 'w') as f:
    json.dump(network_data, f, indent=2)

print("Results exported successfully!")
print("- model_results.csv")
print("- agent_results.csv")
print("- network_structure.json")